In [1]:
import pandas as pd
import numpy as np

import os

os.chdir('results')

In [2]:
def get_metric(dataset, steps=30, eps=0.03, col='ACC'):
    result = pd.DataFrame()
    
    models = os.listdir(dataset)
    
    for model in models:
        try:
            data = pd.read_csv(dataset + '/' + model + f'/attack/fgsm_attackwithout_reg/aa_res_{dataset}_0.csv', index_col=0)
            data.columns = list(map(lambda x: x.strip(), data.columns) )

            result.loc[model, dataset] = data[(data['eps']==eps)&(data['n steps']==steps)][col].values[0]
        except:
            print(dataset, model)
    
    return result


def get_all(steps, eps, col='ACC'):
    
    folders = np.array(os.listdir())
    mask = list(map(lambda x: '.' not in x, folders))

    folders = folders[mask]
    
    result = pd.DataFrame()

    for dataset in folders:
        dataset = str(dataset)
        
        curr = get_metric(dataset, steps, eps, col)
        result[dataset] = curr
        
    return result

In [3]:
get_metric(dataset='GunPointMaleVersusFemale', steps=10, eps=0.03, col='HID')

,GunPointMaleVersusFemale
ResidualCNN,0.814168
SeqS4,0.961766
RNNA,0.686518
TS2VecClassifier,0.789800
LSTM,0.787630


In [4]:
# get_all(10, 0.03, 'ACC')

In [5]:
os.listdir()

['ProximalPhalanxOutlineCorrect',
 'Wafer',
 'GunPointMaleVersusFemale',
 'Strawberry',
 'PowerCons',
 'FordA',
 'FreezerRegularTrain',
 'Coffee']

In [6]:
res = pd.DataFrame()

for dataset in os.listdir():

    res_dataset = pd.DataFrame()
    models = os.listdir(dataset)
    
    for model in models:
        
        path = f'{dataset}/{model}/attack'
        res_model = pd.DataFrame()

        try:
            for experiment in os.listdir(path):
                
                if experiment.startswith('fgsm_disc_attack_same_eps'):
                    disc_eps = float(experiment.split('=')[1])

                    exp_path = os.path.join(path, experiment)
                    csv_files = [file for file in os.listdir(exp_path) if '.csv' in file]

                    res_exp = pd.DataFrame()
                    
                    for file in csv_files:
                        df_exp_alpha = pd.read_csv(os.path.join(exp_path, file))

                        df_exp_alpha.columns = [col_name.strip() for col_name in df_exp_alpha.columns]
                        alpha = float(file.split('=')[1][:-4])

                        df_exp_alpha['alpha'] = alpha
                        res_exp = pd.concat([res_exp, df_exp_alpha])

                    res_exp['disc_eps'] = disc_eps
                    res_model = pd.concat([res_model, res_exp])

        except FileNotFoundError:
            pass
            print(f'No dataset {dataset} model {model}')

        res_model['model'] = model
        res_dataset = pd.concat([res_dataset, res_model])

    res_dataset['dataset'] = dataset
    res = pd.concat([res, res_dataset])

res = res.drop(columns = 'Unnamed: 0')
res = res.reset_index(drop=True)
print(res.shape)
res.head()    

No dataset Wafer model RNNA
No dataset FordA model RNNA
No dataset FreezerRegularTrain model RNNA
No dataset FreezerRegularTrain model LSTM
(57069, 11)


,n steps,ACC,ROC AUC,PR AUC,HID,eps,alpha,disc_eps,model,,dataset
0,1.0,0.896907,0.874945,0.901074,0.005531,0.01,0.001,0.03,ResidualCNN,NaN,ProximalPhalanxOutlineCorrect
1,2.0,0.329897,0.334717,0.629909,0.021204,0.01,0.001,0.03,ResidualCNN,NaN,ProximalPhalanxOutlineCorrect
2,3.0,0.127148,0.160176,0.647605,0.133530,0.01,0.001,0.03,ResidualCNN,NaN,ProximalPhalanxOutlineCorrect
3,4.0,0.068729,0.097007,0.671108,0.178879,0.01,0.001,0.03,ResidualCNN,NaN,ProximalPhalanxOutlineCorrect
4,5.0,0.051546,0.078600,0.680476,0.209370,0.01,0.001,0.03,ResidualCNN,NaN,ProximalPhalanxOutlineCorrect


In [7]:
#res = res[res['eps'] >= res['disc_eps']]

In [8]:
res = res.dropna(axis=1)
res

,n steps,ACC,ROC AUC,PR AUC,HID,eps,alpha,disc_eps,model,dataset
0,1.0,0.896907,0.874945,0.901074,0.005531,0.010,0.001,0.030,ResidualCNN,ProximalPhalanxOutlineCorrect
1,2.0,0.329897,0.334717,0.629909,0.021204,0.010,0.001,0.030,ResidualCNN,ProximalPhalanxOutlineCorrect
2,3.0,0.127148,0.160176,0.647605,0.133530,0.010,0.001,0.030,ResidualCNN,ProximalPhalanxOutlineCorrect
3,4.0,0.068729,0.097007,0.671108,0.178879,0.010,0.001,0.030,ResidualCNN,ProximalPhalanxOutlineCorrect
4,5.0,0.051546,0.078600,0.680476,0.209370,0.010,0.001,0.030,ResidualCNN,ProximalPhalanxOutlineCorrect
...,...,...,...,...,...,...,...,...,...,...
57064,47.0,0.000000,0.000000,0.464286,0.990583,0.015,0.010,0.015,TS2VecClassifier,Coffee
57065,48.0,0.000000,0.000000,0.464286,0.992223,0.015,0.010,0.015,TS2VecClassifier,Coffee
57066,49.0,0.000000,0.000000,0.464286,0.993398,0.015,0.010,0.015,TS2VecClassifier,Coffee
57067,50.0,0.000000,0.000000,0.464286,0.994392,0.015,0.010,0.015,TS2VecClassifier,Coffee


In [9]:
res['Effect'] = 1 - res['ACC']
res['Conceal'] = 1 - res['HID']


res['F_Effect_Conceal'] = 2 * res['Effect'] * res['Conceal'] / (res['Effect'] + res['Conceal'])

res['id_line'] = np.arange(len(res))

res.head()

,n steps,ACC,ROC AUC,PR AUC,HID,eps,alpha,disc_eps,model,dataset,Effect,Conceal,F_Effect_Conceal,id_line
0,1.0,0.896907,0.874945,0.901074,0.005531,0.01,0.001,0.03,ResidualCNN,ProximalPhalanxOutlineCorrect,0.103093,0.994469,0.186819,0
1,2.0,0.329897,0.334717,0.629909,0.021204,0.01,0.001,0.03,ResidualCNN,ProximalPhalanxOutlineCorrect,0.670103,0.978796,0.795554,1
2,3.0,0.127148,0.160176,0.647605,0.133530,0.01,0.001,0.03,ResidualCNN,ProximalPhalanxOutlineCorrect,0.872852,0.866470,0.869649,2
3,4.0,0.068729,0.097007,0.671108,0.178879,0.01,0.001,0.03,ResidualCNN,ProximalPhalanxOutlineCorrect,0.931271,0.821121,0.872734,3
4,5.0,0.051546,0.078600,0.680476,0.209370,0.01,0.001,0.03,ResidualCNN,ProximalPhalanxOutlineCorrect,0.948454,0.790630,0.862381,4


In [10]:
idx = res.groupby(['dataset', 'model'])['F_Effect_Conceal'].idxmax()
res_best = res.loc[idx.values, :].reset_index(drop=True)
res_best

,n steps,ACC,ROC AUC,PR AUC,HID,eps,alpha,disc_eps,model,dataset,Effect,Conceal,F_Effect_Conceal,id_line
0,8.0,0.000000,0.000000,0.464286,9.874950e-02,0.010,0.040,0.010,RNNA,Coffee,1.000000,0.901250,0.948061,56515
1,49.0,0.000000,0.000000,0.464286,3.875074e-03,0.015,0.500,0.015,ResidualCNN,Coffee,1.000000,0.996125,0.998059,52731
2,45.0,0.000000,0.000000,0.464286,4.983097e-17,0.050,0.001,0.015,SeqS4,Coffee,1.000000,1.000000,1.000000,53849
3,51.0,0.000000,0.000000,0.464286,1.369503e-02,0.015,0.100,0.015,TS2VecClassifier,Coffee,1.000000,0.986305,0.993105,56864
4,46.0,0.034091,0.033040,0.484091,2.854579e-03,0.015,0.010,0.030,LSTM,FordA,0.965909,0.997145,0.981279,43344
5,25.0,0.547727,0.547534,0.510199,3.826145e-01,0.010,0.100,0.005,ResidualCNN,FordA,0.452273,0.617385,0.522086,37611
6,51.0,0.072727,0.071981,0.462931,7.913984e-02,0.020,0.500,0.200,SeqS4,FordA,0.927273,0.920860,0.924055,40085
7,36.0,0.101515,0.100460,0.456086,1.527832e-01,0.010,0.100,0.025,TS2VecClassifier,FordA,0.898485,0.847217,0.872098,40427
8,3.0,0.359298,0.359298,0.487340,2.021842e-01,0.010,0.100,0.030,ResidualCNN,FreezerRegularTrain,0.640702,0.797816,0.710679,45698
9,51.0,0.021404,0.021404,0.492852,1.363833e-01,0.100,1.000,0.030,SeqS4,FreezerRegularTrain,0.978596,0.863617,0.917518,49367


In [13]:
res_best.pivot(columns=['dataset'], index='model', values=['Effect', 'Conceal'])

Effect                                \
dataset          Coffee     FordA FreezerRegularTrain   
model                                                   
LSTM                NaN  0.965909                 NaN   
RNNA                1.0       NaN                 NaN   
ResidualCNN         1.0  0.452273            0.640702   
SeqS4               1.0  0.927273            0.978596   
TS2VecClassifier    1.0  0.898485            0.983158   

                                                     \
dataset          GunPointMaleVersusFemale PowerCons   
model                                                 
LSTM                             0.515823  0.744444   
RNNA                                  NaN  0.194444   
ResidualCNN                      0.689873  0.627778   
SeqS4                            0.686709  0.377778   
TS2VecClassifier                 0.490506  0.766667   

                                                                      Conceal  \
dataset          ProximalPhalanxOutlineCorrect Strawberry     Wafer    Coffee   
model                                                                           
LSTM                                  0.996564        NaN  0.607722       NaN   
RNNA                                  0.986254   0.910811       NaN  0.901250   
ResidualCNN                           0.989691   0.535135  0.966580  0.996125   
SeqS4                                 1.000000   0.916216  0.690461  1.000000   
TS2VecClassifier                      1.000000   0.832432  0.838254  0.986305   

                                                                         \
dataset              FordA FreezerRegularTrain GunPointMaleVersusFemale   
model                                                                     
LSTM              0.997145                 NaN                 0.545221   
RNNA                   NaN                 NaN                      NaN   
ResidualCNN       0.617385            0.797816                 0.629451   
SeqS4             0.920860            0.863617                 1.000000   
TS2VecClassifier  0.847217            0.857977                 0.893253   

                                                                               
dataset          PowerCons ProximalPhalanxOutlineCorrect Strawberry     Wafer  
model                                                                          
LSTM              0.825748                      0.954432        NaN  0.665037  
RNNA              0.736928                      0.676892   0.999849       NaN  
ResidualCNN       0.860258                      0.891362   0.852942  0.956523  
SeqS4             0.923573                      0.999823   0.990025  0.561287  
TS2VecClassifier  0.868256                      0.937384   0.971056  0.999833